# Statistiques dans le temps

## Paquets utilisés

In [23]:
import numpy as np
import json
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as ipw
from wordcloud import WordCloud
from typing import List

## Regroupement des mots par années

In [24]:
songs = pd.read_json("./datas/songs.json")

songs["year"] = songs["date"].map(lambda x: datetime.datetime.strptime(x, "%Y-%m-%dT%H:%M:%S").year)

songs.sort_values("year", ascending=False, inplace=True)

songs["words"] = songs["lyrics_keywords"].map(lambda x: x.split(" "))

songs = pd.DataFrame(songs.groupby("year")["lyrics_keywords"].apply(list))

songs["lyrics_keywords"] = songs["lyrics_keywords"].str.join(" ")
songs["lyrics_keywords"] = songs["lyrics_keywords"].apply(lambda x: x.split(" "))

songs = songs.reset_index()

songs = songs[songs["year"] > 1900]

songs

,year,lyrics_keywords
2,1998,"[boom, boom, bang, bad, boy, pose, ton, gun, b..."
3,2000,"[hauts-de-seine, alors, comme, ça, on, veut, r..."
4,2001,"[couplet, génération, qui, fait, honte, ses, p..."
5,2004,"[dédicacé, ceux, qui, rappaient, sur, beatbox,..."
6,2005,"[garde, la, pêche, les, mcs, sont, fauchés, pr..."
7,2006,"[hahaha, mmmm, viens, au, fond, la, classe, là..."
8,2009,"[just, get, home, and, then, leave, again, it'..."
9,2011,"[lève, ton, verre, lève, ton, verre, lève, ton..."
10,2012,"[décembre, aux, abymes, suis, né, une, famille..."
11,2013,"[tous, ces, haineux, ai, amour, pour, vous, as..."


## Affichages de l'utilisation d'un mot par années

In [25]:
# Textbox creation
word_textbox = ipw.Text(value="", placeholder="Search word", description="Word to count:", disabled=False)

songs_count: pd.DataFrame
searched_word: str

def count_word(word: str, words: List[str]):
    words = pd.Series(words)
    return len(words[words == word]) / len(words) * 100

def count_percentage(word: str, words: List[str]):
    words = pd.Series(words)
    return len(words[words == word]) / len(words) * 100

# Function called when the button is clicked
def textbox_changed(word: str):
    searched_word = word
    word_searched = word.lower().strip()

    songs_count = songs
    songs_count["word_count"] = songs_count["lyrics_keywords"].map(lambda x: count_word(word, x))
    songs_count["percentage_of_usage"] = songs_count["lyrics_keywords"].map(lambda x: count_word(word, x))

    songs_count.plot(kind="bar", x="year", y="percentage_of_usage", figsize=(14, 7))

# Button interaction
ipw.interact_manual(textbox_changed, word=word_textbox.value)

interactive(children=(Text(value='', description='word'), Button(description='Run Interact', style=ButtonStyle…

<function __main__.textbox_changed(word: str)>